In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FirstSparkSessionApp").getOrCreate()

24/12/06 16:21:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.format('csv').load('data/2015-summary.csv', inferSchema=True, header=True)

In [3]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,IntegerType,true)))

In [4]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [5]:
df.take(1)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)]

In [6]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [7]:
df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [9]:
df.count()

256

### dropDuplicates()

In [10]:
df_dup = df.select('DEST_COUNTRY_NAME').dropDuplicates()
df_dup.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [11]:
df_dup = df.select('DEST_COUNTRY_NAME').dropDuplicates().cache()
df_dup.show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



In [12]:
df_dup.count()

132

In [13]:
df.sort('DEST_COUNTRY_NAME').show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Algeria|      United States|    4|
|              Angola|      United States|   15|
|            Anguilla|      United States|   41|
| Antigua and Barbuda|      United States|  126|
|           Argentina|      United States|  180|
|               Aruba|      United States|  346|
|           Australia|      United States|  329|
|             Austria|      United States|   62|
|          Azerbaijan|      United States|   21|
|             Bahrain|      United States|   19|
|            Barbados|      United States|  154|
|             Belgium|      United States|  259|
|              Belize|      United States|  188|
|             Bermuda|      United States|  183|
|             Bolivia|      United States|   30|
|Bonaire, Sint Eus...|      United States|   58|
|              Brazil|      United States|  853|
|British Virgin Is..

### expr()

In [15]:
from pyspark.sql.functions import expr

df2 = df.withColumn('withInCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [16]:
df2.show()

+--------------------+-------------------+-----+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withInCountry|
+--------------------+-------------------+-----+-------------+
|       United States|            Romania|   15|        false|
|       United States|            Croatia|    1|        false|
|       United States|            Ireland|  344|        false|
|               Egypt|      United States|   15|        false|
|       United States|              India|   62|        false|
|       United States|          Singapore|    1|        false|
|       United States|            Grenada|   62|        false|
|          Costa Rica|      United States|  588|        false|
|             Senegal|      United States|   40|        false|
|             Moldova|      United States|    1|        false|
|       United States|       Sint Maarten|  325|        false|
|       United States|   Marshall Islands|   39|        false|
|              Guyana|      United States|   64|       

In [17]:
df3 = df.withColumn('category', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME'))

In [18]:
df3.show()

+--------------------+-------------------+-----+--------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|
+--------------------+-------------------+-----+--------+
|       United States|            Romania|   15|   false|
|       United States|            Croatia|    1|   false|
|       United States|            Ireland|  344|   false|
|               Egypt|      United States|   15|   false|
|       United States|              India|   62|   false|
|       United States|          Singapore|    1|   false|
|       United States|            Grenada|   62|   false|
|          Costa Rica|      United States|  588|   false|
|             Senegal|      United States|   40|   false|
|             Moldova|      United States|    1|   false|
|       United States|       Sint Maarten|  325|   false|
|       United States|   Marshall Islands|   39|   false|
|              Guyana|      United States|   64|   false|
|               Malta|      United States|    1|   false|
|            A

In [20]:
df4 = df.withColumn('category', expr('CASE WHEN count < 10 THEN "under" WHEN count >=10 THEN "upper" END'))

In [22]:
df2.groupBy('withInCountry').count().show()

+-------------+-----+
|withInCountry|count|
+-------------+-----+
|         true|    1|
|        false|  255|
+-------------+-----+



In [23]:
df3.groupBy('category').count().show()

+--------+-----+
|category|count|
+--------+-----+
|    true|    1|
|   false|  255|
+--------+-----+



### spark.sql

In [8]:
df.createOrReplaceTempView('mobility_data')

In [10]:
spark.sql("select * from mobility_data").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [25]:
# Projection, filter
df5 = df.where('count<5')
df5.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Croatia|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|               Malta|      United States|    1|
|             Algeria|      United States|    4|
|       United States|          Gibraltar|    1|
|Saint Vincent and...|      United States|    1|
|            Suriname|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|           Malaysia|    3|
|            Thailand|      United States|    3|
|             Liberia|      United States|    2|
|             Hungary|      United States|    2|
|       United States|            Vietnam|    2|
|        Burkina Faso|      United States|    1|
|            Djibouti|      United States|    1|
|       United States|            Estonia|    1|
|       United State

In [26]:
df5.count()

46

In [32]:
df7 = df.where("count < 5").where("ORIGIN_COUNTRY_NAME <> 'United States'")
df7.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|           Malaysia|    3|
|    United States|            Vietnam|    2|
|    United States|            Estonia|    1|
|    United States|            Hungary|    3|
|    United States|           Thailand|    4|
|    United States|            Liberia|    2|
|    United States|              Malta|    2|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|          Greenland|    4|
|    United States|          Indonesia|    2|
|    United States|         Monten

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SecondSparkSessionApp").getOrCreate()

24/12/09 10:38:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.format('csv').option('header', 'true')\
            .option('inferSchema','true')\
            .load('data/emp.csv')

In [4]:
df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [6]:
df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [5]:
df.select('ename', 'deptno').show()

+------+------+
| ename|deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [7]:
df.select('ename','deptno').where('deptno=20').show()

+-----+------+
|ename|deptno|
+-----+------+
|SMITH|    20|
|JONES|    20|
|SCOTT|    20|
|ADAMS|    20|
| FORD|    20|
+-----+------+



In [8]:
from pyspark.sql.functions import count

df.select(count('job')).show()

+----------+
|count(job)|
+----------+
|        15|
+----------+



In [9]:
# null 포함해서 카운트
df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [10]:
df.select('job').distinct().show()

+---------+
|      job|
+---------+
|  ANALYST|
| SALESMAN|
|    CLERK|
|  MANAGER|
|PRESIDENT|
+---------+



In [11]:
df.select('job').distinct().count()

5

In [16]:
from pyspark.sql.functions import countDistinct,approx_count_distinct

df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [19]:
df.select(approx_count_distinct('job',0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [20]:
from pyspark.sql.functions import *

df.select(first('ename'), last('ename')).show()

+------------+-----------+
|first(ename)|last(ename)|
+------------+-----------+
|       SMITH|       JACK|
+------------+-----------+



In [22]:
df.select(count('empno'),count('*'),max('ename'),min('ename')).show()

+------------+--------+----------+----------+
|count(empno)|count(1)|max(ename)|min(ename)|
+------------+--------+----------+----------+
|          15|      15|      WARD|     ADAMS|
+------------+--------+----------+----------+



In [21]:
df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [23]:
df.selectExpr('sum(distinct(sal))').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [24]:
df.selectExpr('avg(distinct(sal))').show()

+-----------------+
|avg(DISTINCT sal)|
+-----------------+
|2151.923076923077|
+-----------------+



In [26]:
df.select(count('sal').alias('total_count'),
         sum('sal').alias('total_salary'),
         avg('sal').alias('avg_salary'),
         expr('mean(sal)').alias('mean_salary')).show()

+-----------+------------+------------------+------------------+
|total_count|total_salary|        avg_salary|       mean_salary|
+-----------+------------+------------------+------------------+
|         15|       32225|2148.3333333333335|2148.3333333333335|
+-----------+------------+------------------+------------------+



## groupBy()

In [27]:
df

DataFrame[empno: int, ename: string, job: string, mgr: int, hiredate: string, sal: int, comm: int, deptno: int]

In [29]:
df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



## agg()

In [31]:
df.groupBy('job').agg(expr('avg(sal) as SAL_AVG')).show()

+---------+------------------+
|      job|           SAL_AVG|
+---------+------------------+
|  ANALYST|            3000.0|
| SALESMAN|            1400.0|
|    CLERK|            1470.0|
|  MANAGER|2758.3333333333335|
|PRESIDENT|            5000.0|
+---------+------------------+



### 모집단 전체 표준편차

In [33]:
df.groupBy('job').agg(expr('stddev_pop(sal) as SAL_AVG')).show()

+---------+------------------+
|      job|           SAL_AVG|
+---------+------------------+
|  ANALYST|               0.0|
| SALESMAN|154.11035007422439|
|    CLERK| 880.6815542521599|
|  MANAGER|223.91714737574006|
|PRESIDENT|               0.0|
+---------+------------------+



### 표본 표준편차

In [32]:
df.groupBy('job').agg(expr('std(sal) as SAL_AVG')).show()

+---------+------------------+
|      job|           SAL_AVG|
+---------+------------------+
|  ANALYST|               0.0|
| SALESMAN|177.95130420052183|
|    CLERK| 984.6319109189992|
|  MANAGER|274.24137786507225|
|PRESIDENT|              null|
+---------+------------------+



## 윈도우 함수
1. 순위, 정렬 : rank(), row_number(), dense_rank()
2. 누계 : sum(), avg(), max(), min() + over()
3. 이동평균 : over + rowsBetween, rangeBetween
4. 시차, 선형 : lag, lead

In [34]:
from pyspark.sql.functions import desc
from pyspark.sql.window import Window

windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [35]:
df.withColumn('salary_rank', salAllRank).show()

24/12/09 13:39:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|null|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|         10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|         11|
| 7654|MARTIN| SALES

In [36]:
df.select('empno', salAllRank.alias('salary_rank')).show()

+-----+-----------+
|empno|salary_rank|
+-----+-----------+
| 7839|          1|
| 9292|          2|
| 7788|          3|
| 7902|          3|
| 7566|          5|
| 7698|          6|
| 7782|          7|
| 7499|          8|
| 7844|          9|
| 7934|         10|
| 7521|         11|
| 7654|         11|
| 7876|         13|
| 7900|         14|
| 7369|         15|
+-----+-----------+



24/12/09 13:45:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [37]:
windowspec = Window.partitionBy('job').orderBy(desc('sal'))
windowspec

In [39]:
df.select('job', 'ename', 'sal', rank().over(windowspec).alias('salJobRank')).show()

+---------+------+----+----------+
|      job| ename| sal|salJobRank|
+---------+------+----+----------+
|  ANALYST| SCOTT|3000|         1|
|  ANALYST|  FORD|3000|         1|
| SALESMAN| ALLEN|1600|         1|
| SALESMAN|TURNER|1500|         2|
| SALESMAN|  WARD|1250|         3|
| SALESMAN|MARTIN|1250|         3|
|    CLERK|  JACK|3200|         1|
|    CLERK|MILLER|1300|         2|
|    CLERK| ADAMS|1100|         3|
|    CLERK| JAMES| 950|         4|
|    CLERK| SMITH| 800|         5|
|  MANAGER| JONES|2975|         1|
|  MANAGER| BLAKE|2850|         2|
|  MANAGER| CLARK|2450|         3|
|PRESIDENT|  KING|5000|         1|
+---------+------+----+----------+



In [40]:
df

DataFrame[empno: int, ename: string, job: string, mgr: int, hiredate: string, sal: int, comm: int, deptno: int]

In [42]:
windowspec = Window.partitionBy('deptno').orderBy(desc('sal'))
df.select('deptno', 'sal', rank().over(windowspec).alias('deptSalRank')).show()

+------+----+-----------+
|deptno| sal|deptSalRank|
+------+----+-----------+
|    20|3000|          1|
|    20|3000|          1|
|    20|2975|          3|
|    20|1100|          4|
|    20| 800|          5|
|    10|5000|          1|
|    10|2450|          2|
|    10|1300|          3|
|    70|3200|          1|
|    30|2850|          1|
|    30|1600|          2|
|    30|1500|          3|
|    30|1250|          4|
|    30|1250|          4|
|    30| 950|          6|
+------+----+-----------+



In [46]:
df1 = df.withColumn('dept_salary_rank',row_number().over(windowspec))
df1.select('ename','deptno','sal','dept_salary_rank').show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               2|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
|  JACK|    70|3200|               1|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               5|
| JAMES|    30| 950|               6|
+------+------+----+----------------+



In [48]:
# 부서별 급여 누적 합계
windowspec = Window.partitionBy('deptno').orderBy('empno')
df2 = df.withColumn('cum_salary', sum('sal').over(windowspec))
df2.select('ename','deptno','sal','cum_salary').show()

+------+------+----+----------+
| ename|deptno| sal|cum_salary|
+------+------+----+----------+
| SMITH|    20| 800|       800|
| JONES|    20|2975|      3775|
| SCOTT|    20|3000|      6775|
| ADAMS|    20|1100|      7875|
|  FORD|    20|3000|     10875|
| CLARK|    10|2450|      2450|
|  KING|    10|5000|      7450|
|MILLER|    10|1300|      8750|
|  JACK|    70|3200|      3200|
| ALLEN|    30|1600|      1600|
|  WARD|    30|1250|      2850|
|MARTIN|    30|1250|      4100|
| BLAKE|    30|2850|      6950|
|TURNER|    30|1500|      8450|
| JAMES|    30| 950|      9400|
+------+------+----+----------+



In [49]:
# 부서별 평균 급여
windowspec = Window.partitionBy('deptno')
df3 = df.withColumn('dept_avg_sal', avg('sal').over(windowspec))
df3.select('deptno','sal','dept_avg_sal').show()

+------+----+------------------+
|deptno| sal|      dept_avg_sal|
+------+----+------------------+
|    20| 800|            2175.0|
|    20|2975|            2175.0|
|    20|3000|            2175.0|
|    20|1100|            2175.0|
|    20|3000|            2175.0|
|    10|2450|2916.6666666666665|
|    10|5000|2916.6666666666665|
|    10|1300|2916.6666666666665|
|    70|3200|            3200.0|
|    30|1600|1566.6666666666667|
|    30|1250|1566.6666666666667|
|    30|1250|1566.6666666666667|
|    30|2850|1566.6666666666667|
|    30|1500|1566.6666666666667|
|    30| 950|1566.6666666666667|
+------+----+------------------+



In [51]:
df.groupBy('deptno').agg(avg('sal').alias('dept_avg_sal')).show()

+------+------------------+
|deptno|      dept_avg_sal|
+------+------------------+
|    20|            2175.0|
|    10|2916.6666666666665|
|    70|            3200.0|
|    30|1566.6666666666667|
+------+------------------+



In [56]:
# lag: 현재 행 기준으로 이전 행의 값을 가져옴
# lead: 현재 행 기준으로 다음 행의 값을 가져옴

windowspec = Window.partitionBy('deptno').orderBy('empno')
df4 = df.withColumn('prev_salary', lag('sal').over(windowspec))\
        .withColumn('next_salary', lead('sal').over(windowspec))
df4.select('deptno','sal','prev_salary','next_salary').show()

+------+----+-----------+-----------+
|deptno| sal|prev_salary|next_salary|
+------+----+-----------+-----------+
|    20| 800|       null|       2975|
|    20|2975|        800|       3000|
|    20|3000|       2975|       1100|
|    20|1100|       3000|       3000|
|    20|3000|       1100|       null|
|    10|2450|       null|       5000|
|    10|5000|       2450|       1300|
|    10|1300|       5000|       null|
|    70|3200|       null|       null|
|    30|1600|       null|       1250|
|    30|1250|       1600|       1250|
|    30|1250|       1250|       2850|
|    30|2850|       1250|       1500|
|    30|1500|       2850|        950|
|    30| 950|       1500|       null|
+------+----+-----------+-----------+



In [57]:
df.groupBy('deptno','job').agg(count('*'),sum('sal')).show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    20|  ANALYST|       2|    6000|
|    20|  MANAGER|       1|    2975|
|    30|  MANAGER|       1|    2850|
|    70|    CLERK|       1|    3200|
|    30| SALESMAN|       4|    5600|
|    30|    CLERK|       1|     950|
|    20|    CLERK|       2|    1900|
|    10|PRESIDENT|       1|    5000|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
+------+---------+--------+--------+



In [58]:
# rollup
df.rollup('deptno','job').agg(count('*'),sum('sal')).orderBy('deptno','job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [60]:
df.rollup('deptno','job').agg(max('sal'), min('sal')).orderBy('deptno','job').show()

+------+---------+--------+--------+
|deptno|      job|max(sal)|min(sal)|
+------+---------+--------+--------+
|  null|     null|    5000|     800|
|    10|     null|    5000|    1300|
|    10|    CLERK|    1300|    1300|
|    10|  MANAGER|    2450|    2450|
|    10|PRESIDENT|    5000|    5000|
|    20|     null|    3000|     800|
|    20|  ANALYST|    3000|    3000|
|    20|    CLERK|    1100|     800|
|    20|  MANAGER|    2975|    2975|
|    30|     null|    2850|     950|
|    30|    CLERK|     950|     950|
|    30|  MANAGER|    2850|    2850|
|    30| SALESMAN|    1600|    1250|
|    70|     null|    3200|    3200|
|    70|    CLERK|    3200|    3200|
+------+---------+--------+--------+



In [59]:
# cube
df.cube('deptno','job').agg(count('*'), sum('sal')).orderBy('deptno','job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  null|     null|      15|   32225|
|  null|  ANALYST|       2|    6000|
|  null|    CLERK|       5|    7350|
|  null|  MANAGER|       3|    8275|
|  null|PRESIDENT|       1|    5000|
|  null| SALESMAN|       4|    5600|
|    10|     null|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     null|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     null|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     null|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [62]:
df.cube('deptno', 'job') \
    .agg(
        count('*').alias('count'),
        round(avg('sal'), 2).alias('avg_sal'),
        max('sal').alias('max_sal')
    ) \
    .orderBy('deptno', 'job') \
    .select('deptno', 'job', 'avg_sal', 'max_sal') \
    .show()

+------+---------+-------+-------+
|deptno|      job|avg_sal|max_sal|
+------+---------+-------+-------+
|  null|     null|2148.33|   5000|
|  null|  ANALYST| 3000.0|   3000|
|  null|    CLERK| 1470.0|   3200|
|  null|  MANAGER|2758.33|   2975|
|  null|PRESIDENT| 5000.0|   5000|
|  null| SALESMAN| 1400.0|   1600|
|    10|     null|2916.67|   5000|
|    10|    CLERK| 1300.0|   1300|
|    10|  MANAGER| 2450.0|   2450|
|    10|PRESIDENT| 5000.0|   5000|
|    20|     null| 2175.0|   3000|
|    20|  ANALYST| 3000.0|   3000|
|    20|    CLERK|  950.0|   1100|
|    20|  MANAGER| 2975.0|   2975|
|    30|     null|1566.67|   2850|
|    30|    CLERK|  950.0|    950|
|    30|  MANAGER| 2850.0|   2850|
|    30| SALESMAN| 1400.0|   1600|
|    70|     null| 3200.0|   3200|
|    70|    CLERK| 3200.0|   3200|
+------+---------+-------+-------+



In [63]:
spark.stop()